In [1]:
import openmc
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Function to run OpenMC and extract k-effective
def run_openmc_simulation(N, batches=100, inactive=10, particles=10000):
    """
    Runs OpenMC with given settings and measures runtime.

    Parameters:
    - N (int): TRISO lattice size (N x N x N)
    - batches (int): Number of batches
    - inactive (int): Number of inactive batches
    - particles (int): Number of particles per batch

    Returns:
    - k_values (list): k-effective values over batches
    - runtime (float): Total runtime in seconds
    """

    # Load existing geometry and materials
    materials = openmc.Materials.from_xml("materials.xml")
    geometry = openmc.Geometry.from_xml("geometry.xml")

    # Define settings
    settings = openmc.Settings()
    settings.batches = batches
    settings.inactive = inactive
    settings.particles = particles
    settings.run_mode = 'eigenvalue'

    # Define a neutron source (uniform within core)
    source = openmc.Source()
    source.space = openmc.stats.Box(lower_left=(-50, -50, -50), upper_right=(50, 50, 50))
    source.energy = openmc.stats.Discrete([2.0e6], [1.0])  # Fast fission spectrum (2 MeV)
    settings.source = source

    settings.export_to_xml()

    # Record start time
    start_time = time.time()

    # Run OpenMC
    openmc.run()

    # Measure runtime
    runtime = time.time() - start_time

    # Extract k-effective from statepoint
    sp = openmc.StatePoint("statepoint.100.h5")  # Use last batch
    k_values = [k.nominal_value for k in sp.k_generation]  # Extract k over batches

    return k_values, runtime


# ---- Study: Optimize k vs. batches ---- #
batch_sizes = [50, 100, 200, 300]
k_results = {}

for batches in batch_sizes:
    print(f"Running k-eigenvalue calculation for {batches} batches...")
    k_values, _ = run_openmc_simulation(N=16, batches=batches, inactive=10, particles=10000)
    k_results[batches] = k_values

# Plot k-effective vs. batches
plt.figure(figsize=(8, 6))
for batches, k_values in k_results.items():
    plt.plot(range(len(k_values)), k_values, label=f"{batches} batches")

plt.xlabel("Batch Number")
plt.ylabel("k-effective")
plt.title("Convergence of k-effective vs. Batches")
plt.legend()
plt.grid()
plt.show()


# ---- Study: Optimize TRISO Lattice Size (N) ---- #
lattice_sizes = [8, 12, 16, 20, 24]
runtime_results = []

for N in lattice_sizes:
    print(f"Running for TRISO lattice size {N}x{N}x{N}...")
    _, runtime = run_openmc_simulation(N=N, batches=100, inactive=10, particles=10000)
    runtime_results.append(runtime)

# Plot runtime vs. TRISO lattice size
plt.figure(figsize=(8, 6))
plt.plot(lattice_sizes, runtime_results, marker='o', linestyle='-')
plt.xlabel("TRISO Lattice Size (N x N x N)")
plt.ylabel("Runtime (seconds)")
plt.title("Simulation Runtime vs. TRISO Lattice Size")
plt.grid()
plt.show()

# Display results in a table
runtime_df = pd.DataFrame({"Lattice Size (N)": lattice_sizes, "Runtime (s)": runtime_results})
import ace_tools as tools
tools.display_dataframe_to_user(name="TRISO Lattice Runtime Study", dataframe=runtime_df)


ModuleNotFoundError: No module named 'openmc'